## OMOP Attribute Study
Purpose: This notebook merges two datasets containing OMOP field definitions


- omop_gpt.csv: Contains OMOP field definitions from GPT
- omop_fields_claude: Contains OMOP field definitions from Claude

The datasets are merged on the common field identifier column to create a comprehensive
mapping of OMOP fields with information from both sources.

In [10]:
import pandas as pd

In [11]:
DATA_INPUT_PATH = "/home/paula/Documentos/pmorenofajardo87/NOCTUAMED/noctuamed_rafa/transmisiones_a_csv/transmisiones_anonimizadas_gem/estandarizados/omop_standardized_data.csv"

In [12]:
omop_gpt = pd.read_csv("OMOP_fields_gpt.csv")
omop_claude = pd.read_csv("OMOP_fields_claude.csv")
omop_standardized_data = pd.read_csv(DATA_INPUT_PATH)

In [13]:
# Rename the first column in omop_claude to match omop_gpt for merging
omop_claude = omop_claude.rename(columns={"Campo": "OMOP Field"})

# Merge the two dataframes on the "OMOP Field" column
merged_df = pd.merge(
    omop_gpt, 
    omop_claude, 
    on="OMOP Field", 
    how="outer",  # Use outer join to keep all rows from both dataframes
    suffixes=('_gpt', '_claude')  # Add suffixes to distinguish columns with the same name
)

# Display the first few rows of the merged dataframe
print(f"Merged dataframe shape: {merged_df.shape}")
merged_df.head()

Merged dataframe shape: (81, 10)


,OMOP Field,Descripción_gpt,Tabla OMOP sugerida,Tipo de dato_gpt,Rango esperado / Observaciones,Significado,Tipo de dato_claude,Unidad,Descripción_claude,Observaciones
0,CLINIC_NAME,Nombre de la clínica,VISIT_OCCURRENCE,string,Nombre del centro médico,Nombre de la clínica,Varchar,NaN,Centro médico donde se realiza seguimiento,Ubicación de la consulta o seguimiento
1,MANUFACTURER_PRIORITY,Prioridad del fabricante,DEVICE_EXPOSURE,string / ordinal,Interno; sin rango definido,Prioridad del fabricante,Varchar/Integer,NaN,Nivel de prioridad según fabricante,Puede indicar urgencia de revisión
2,MDC_IDC_DEV_IMPLANT_DT,Fecha de implantación,DEVICE_EXPOSURE,date,Fecha válida,Fecha de implante,Date,NaN,Fecha en que se implantó el dispositivo,Importante para calcular longevidad
3,MDC_IDC_DEV_MFG,NaN,NaN,NaN,NaN,Fabricante del dispositivo,Varchar,NaN,Empresa fabricante del dispositivo,"Ej: ""Medtronic"", ""Boston Scientific"", ""Abbott"""
4,MDC_IDC_DEV_MODEL,NaN,NaN,NaN,NaN,Modelo del dispositivo,Varchar,NaN,Modelo específico del dispositivo,Código o nombre del modelo comercial


In [14]:
# Save the merged dataframe to a new CSV file
merged_df.to_csv("OMOP_fields_merged.csv", index=False, header=True)
print("Merged dataframe saved to 'OMOP_fields_merged.csv'")

Merged dataframe saved to 'OMOP_fields_merged.csv'


In [15]:
# Analyze the merge results
print(f"Number of rows in omop_gpt: {len(omop_gpt)}")
print(f"Number of rows in omop_claude: {len(omop_claude)}")
print(f"Number of rows in merged_df: {len(merged_df)}")

# Check which fields are in one dataset but not the other
gpt_only = set(omop_gpt["OMOP Field"]) - set(omop_claude["OMOP Field"])
claude_only = set(omop_claude["OMOP Field"]) - set(omop_gpt["OMOP Field"])
common_fields = set(omop_gpt["OMOP Field"]) & set(omop_claude["OMOP Field"])

print(f"\nNumber of fields only in GPT dataset: {len(gpt_only)}")
print(f"Number of fields only in Claude dataset: {len(claude_only)}")
print(f"Number of fields common to both datasets: {len(common_fields)}")

# Display fields that are only in one dataset
if len(gpt_only) > 0:
    print("\nFields only in GPT dataset:")
    for field in sorted(gpt_only):
        print(f"- {field}")

if len(claude_only) > 0:
    print("\nFields only in Claude dataset:")
    for field in sorted(claude_only):
        print(f"- {field}")

Number of rows in omop_gpt: 61
Number of rows in omop_claude: 80
Number of rows in merged_df: 81

Number of fields only in GPT dataset: 0
Number of fields only in Claude dataset: 20
Number of fields common to both datasets: 60

Fields only in Claude dataset:
- MDC_IDC_DEV_MFG
- MDC_IDC_DEV_MODEL
- MDC_IDC_DEV_SERIAL
- MDC_IDC_DEV_TYPE
- MDC_IDC_MSMT_BATTERY_DTM
- MDC_IDC_MSMT_BATTERY_REMAINING_LONGEVITY
- MDC_IDC_MSMT_BATTERY_RRT_TRIGGER
- MDC_IDC_MSMT_BATTERY_STATUS
- MDC_IDC_MSMT_BATTERY_VOLTAGE
- MDC_IDC_MSMT_LEADCHNL_LV_IMPEDANCE_VALUE
- MDC_IDC_MSMT_LEADCHNL_RV_IMPEDANCE_VALUE
- MDC_IDC_SESS_DTM
- MDC_IDC_SESS_TYPE
- MDC_IDC_SET_BRADY_AV_DELAY_HIGH
- MDC_IDC_SET_BRADY_AV_DELAY_LOW
- MDC_IDC_SET_BRADY_HYSTRATE
- MDC_IDC_SET_BRADY_MAX_SENSOR_RATE
- MDC_IDC_SET_BRADY_MAX_TRACKING_RATE
- MDC_IDC_SET_BRADY_MODE
- MDC_IDC_STAT_TACHYTHERAPY_SHOCKS_DELIVERED_RECENT


In [16]:

print("data types in gpt")
print(set(merged_df['Tipo de dato_gpt'].to_list()))
print("data types in claude")
print(set(merged_df['Tipo de dato_claude'].to_list()))
print("number of fields data types  recognyzed by chat gpt:")
print(len(merged_df['Tipo de dato_gpt'].dropna()))
print("number of fields data types  recognyzed by chat gpt:")
print(len(merged_df['Tipo de dato_claude'].dropna()))

data types in gpt
{'numeric', 'date', 'string / ordinal', nan, 'string / concept_id', 'integer', 'string', 'datetime'}
data types in claude
{'Varchar/Integer', 'Varchar', 'Integer', 'Datetime', 'Boolean', 'Date', 'Numeric'}
number of fields data types  recognyzed by chat gpt:
61
number of fields data types  recognyzed by chat gpt:
81


In [17]:
merged_df[merged_df['Tipo de dato_claude'] == 'Varchar']

,OMOP Field,Descripción_gpt,Tabla OMOP sugerida,Tipo de dato_gpt,Rango esperado / Observaciones,Significado,Tipo de dato_claude,Unidad,Descripción_claude,Observaciones
0,CLINIC_NAME,Nombre de la clínica,VISIT_OCCURRENCE,string,Nombre del centro médico,Nombre de la clínica,Varchar,NaN,Centro médico donde se realiza seguimiento,Ubicación de la consulta o seguimiento
3,MDC_IDC_DEV_MFG,NaN,NaN,NaN,NaN,Fabricante del dispositivo,Varchar,NaN,Empresa fabricante del dispositivo,"Ej: ""Medtronic"", ""Boston Scientific"", ""Abbott"""
4,MDC_IDC_DEV_MODEL,NaN,NaN,NaN,NaN,Modelo del dispositivo,Varchar,NaN,Modelo específico del dispositivo,Código o nombre del modelo comercial
5,MDC_IDC_DEV_SERIAL,NaN,NaN,NaN,NaN,Número de serie,Varchar,NaN,Número único de identificación del dispositivo,Identificador único del fabricante
6,MDC_IDC_DEV_TYPE,NaN,NaN,NaN,NaN,Tipo de dispositivo,Varchar,NaN,Categoría del dispositivo implantado,"Ej: ""ICD"", ""CRT-D"", ""Pacemaker"", ""CRT-P"""
11,MDC_IDC_MSMT_BATTERY_STATUS,NaN,NaN,NaN,NaN,Estado de la batería,Varchar,NaN,Estado actual de la batería,"Ej: ""Normal"", ""ERI"" (Elective Replacement Indi..."
14,MDC_IDC_MSMT_LEADCHNL_LV_LEAD_CHANNEL_STATUS,Estado canal LV,OBSERVATION,string,"OK, fallo, alerta",Estado del electrodo ventricular izquierdo,Varchar,NaN,Estado funcional del electrodo ventricular izq...,"Ej: ""Normal"", ""Noise"", ""Fracture"""
19,MDC_IDC_MSMT_LEADCHNL_RA_LEAD_CHANNEL_STATUS,Estado canal RA,OBSERVATION,string,"OK, fallo, alerta",Estado del electrodo auricular derecho,Varchar,NaN,Estado funcional del electrodo auricular,"Ej: ""Normal"", ""Noise"", ""Fracture"""
24,MDC_IDC_MSMT_LEADCHNL_RV_LEAD_CHANNEL_STATUS,Estado canal RV,OBSERVATION,string,"OK, fallo, alerta",Estado del electrodo ventricular derecho,Varchar,NaN,Estado funcional del electrodo ventricular der...,"Ej: ""Normal"", ""Noise"", ""Fracture"""
29,MDC_IDC_SESS_TYPE,NaN,NaN,NaN,NaN,Tipo de sesión,Varchar,NaN,"Categoría de la sesión (presencial, remota)","Ej: ""In-clinic"", ""Remote"", ""Ambulatory"""


In [18]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd

# Lista de campos únicos para el desplegable
field_options = merged_df['OMOP Field'].unique().tolist()

# Crear el desplegable
field_dropdown = widgets.Dropdown(
    options=field_options,
    value='MDC_IDC_DEV_MODEL',
    description='Selecciona un campo:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

# Crear un contenedor para mostrar resultados
output = widgets.Output()

# Función de actualización
def update_field_info(change):
    with output:
        clear_output()  # Limpia la salida anterior
        selected_field = change['new']
        field_info = merged_df[merged_df['OMOP Field'] == selected_field].iloc[0]  # conserva formato DataFrame
        display(field_info)

# Ejecutar la función por defecto una vez
update_field_info({'new': field_dropdown.value})

# Vincular el cambio del desplegable
field_dropdown.observe(update_field_info, names='value')

# Mostrar los widgets
display(field_dropdown, output)


Dropdown(description='Selecciona un campo:', index=4, layout=Layout(width='50%'), options=('CLINIC_NAME', 'MAN…

Output()

In [19]:

manufacturer_code = omop_standardized_data["manufacturer_code"].unique()
samples = []
for i in manufacturer_code:
    sample = omop_standardized_data[omop_standardized_data["manufacturer_code"] == i].sample(3)
    samples.append(sample)
samples_df = pd.concat(samples, ignore_index=True)
display(samples_df)

,source_file,manufacturer_code,CLINIC_NAME,MANUFACTURER_PRIORITY,MDC_IDC_SESS_DTM,MDC_IDC_SESS_TYPE,MDC_IDC_DEV_TYPE,MDC_IDC_DEV_MFG,MDC_IDC_DEV_MODEL,MDC_IDC_DEV_SERIAL,...,MDC_IDC_STAT_TACHYTHERAPY_SHOCKS_DELIVERED_TOTAL,MDC_IDC_STAT_TACHYTHERAPY_SHOCKS_ABORTED_TOTAL,MDC_IDC_STAT_TACHYTHERAPY_ATP_DELIVERED_TOTAL,MDC_IDC_STAT_AT_DTM_START,MDC_IDC_STAT_AT_DTM_END,MDC_IDC_STAT_AT_BURDEN_PERCENT,MDC_IDC_STAT_AT_MODE_SW_COUNT,MDC_IDC_STAT_AT_MODE_SW_MAX_DURATION,patient_id_pid2,patient_id_pid3
0,p1_abbott/85204094.hl7,p1_abbott,NaN,NaN,20250325060035,RemoteScheduled,IPG,Abbott,2272 Assurity MRI(TM),6700726,...,NaN,NaN,NaN,20241214020617,20250325060139,6.0,23.0,219202.0,NaN,"model:2272/serial:6700726, 332044"
1,p1_abbott/84999858.hl7,p1_abbott,NaN,NaN,20250311040016,RemoteScheduled,IPG,Abbott,2272 Assurity MRI(TM),6700726,...,NaN,NaN,NaN,20241214020524,20250311040120,7.0,23.0,219202.0,NaN,"model:2272/serial:6700726, 332044"
2,p1_abbott/85204093.hl7,p1_abbott,NaN,NaN,20250318055721,RemoteScheduled,IPG,Abbott,2272 Assurity MRI(TM),6700726,...,NaN,NaN,NaN,20241214020551,20250318055825,7.0,23.0,219202.0,NaN,"model:2272/serial:6700726, 332044"
3,p2_biotronik/a16aca36-6bc1-4de3-9b48-9519e1bac...,p2_biotronik,Hospital Universitario de Gran Canaria Dr. Negrin,Amarillo,20250117T223153+0000,Remote,IPG,Biotronik,Enitra 6 SR-T,69986066,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,p2_biotronik/04f8bd87-b57c-475f-9253-23ab6dd2a...,p2_biotronik,Hospital Universitario de Gran Canaria Dr. Negrin,Amarillo,20250223T220731+0000,Remote,IPG,Biotronik,Enitra 6 SR-T,1000096403,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,p2_biotronik/02337735-94b9-4022-9a24-549b39e4c...,p2_biotronik,Hospital Universitario de Gran Canaria Dr. Negrin,Amarillo,20250327T004916+0000,Remote,IPG,Biotronik,Epyra 6 SR-T,68948906,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,p3_boston_scientific/1010566883.hl7,p3_boston_scientific,NaN,NaN,20250403215000+0000,NaN,S-ICD,Boston Scientific,EMBLEM MRI S-ICD,140574,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1001412373,"1001412373, 171633"
7,p3_boston_scientific/1010568115.hl7,p3_boston_scientific,NaN,NaN,20250403233400+0000,NaN,DAI,Boston Scientific,VIGILANT EL ICD,619014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1001444771,"1001444771, 563566"
8,p3_boston_scientific/1010568115.hl7,p3_boston_scientific,NaN,NaN,20250403233400+0000,NaN,DAI,Boston Scientific,VIGILANT EL ICD,619014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1001444771,"1001444771, 563566"
9,p4_medtronic/D2177F36-B81B-4359-B883-3A6912DB7...,p4_medtronic,NaN,NaN,20220703083846+0000,Remote,Pacemaker,Medtronic,ADDR01,NWB871343S,...,NaN,NaN,NaN,20220408103245+0000,20220703083846+0000,0.0,6.0,NaN,[],[]


In [20]:
# Date or dateTime OMOP fields
# Filtrar filas donde 'Tipo de dato_claude' o 'Tipo de dato_gpt' contienen 'date', 'datetime' o 'timestamp'
dates_fields = merged_df[
    (merged_df['Tipo de dato_claude'].str.strip().str.lower().isin(['date', 'datetime', 'timestamp'])) |
    (merged_df['Tipo de dato_gpt'].str.strip().str.lower().isin(['date', 'datetime', 'timestamp']))
    ]
dates_fields[['Tipo de dato_gpt','Tipo de dato_claude','Significado','Descripción_claude']]

,Tipo de dato_gpt,Tipo de dato_claude,Significado,Descripción_claude
2,date,Date,Fecha de implante,Fecha en que se implantó el dispositivo
7,NaN,Datetime,Fecha y hora de medición de batería,Momento de la medición del estado de batería
28,NaN,Datetime,Fecha y hora de sesión,Momento exacto de la interrogación del disposi...
55,datetime,Datetime,Fin de período de evaluación AT,Fecha y hora de fin para estadísticas de taqui...
56,datetime,Datetime,Inicio de período de evaluación AT,Fecha y hora de inicio para estadísticas de ta...
69,datetime,Datetime,Fin de período reciente de terapia,Fecha y hora de fin del período reciente evaluado
70,datetime,Datetime,Inicio de período reciente de terapia,Fecha y hora de inicio del período reciente ev...
76,datetime,Datetime,Fin de período total de terapia,Fecha y hora de fin del período total evaluado
77,datetime,Datetime,Inicio de período total de terapia,Fecha y hora de inicio del período total evaluado
78,date,Date,Fecha de nacimiento,Fecha de nacimiento del paciente


parece que claude ha funcionado mejor describiendo los campos fecha y tiempo

In [21]:
dates_omop_fields = dates_fields['OMOP Field'].to_list()
cols_existentes = [col for col in dates_omop_fields if col in samples_df.columns]
samples_df_dates = samples_df[cols_existentes]
print("Dataframe with samples with dates:")
samples_df_dates.sample(10)

Dataframe with samples with dates:


,MDC_IDC_DEV_IMPLANT_DT,MDC_IDC_MSMT_BATTERY_DTM,MDC_IDC_SESS_DTM,MDC_IDC_STAT_AT_DTM_END,MDC_IDC_STAT_AT_DTM_START,MDC_IDC_STAT_TACHYTHERAPY_RECENT_DTM_END,MDC_IDC_STAT_TACHYTHERAPY_RECENT_DTM_START,MDC_IDC_STAT_TACHYTHERAPY_TOTAL_DTM_END,MDC_IDC_STAT_TACHYTHERAPY_TOTAL_DTM_START
13,2024-06-03,NaN,2024-09-02T02:17:49+02:00,NaN,NaN,NaN,NaN,NaN,NaN
6,20210729,NaN,20250403215000+0000,NaN,NaN,NaN,NaN,NaN,NaN
2,20230321000000,20250317144939,20250318055721,20250318055825,20241214020551,2.025032e+13,2.024121e+13,NaN,NaN
11,NaN,20220703083846+0000,20220703083846+0000,20220703083846+0000,20220408103245+0000,NaN,NaN,NaN,NaN
5,20180601T000000,20250327T012600,20250327T004916+0000,NaN,NaN,NaN,NaN,NaN,NaN
8,20220603,NaN,20250403233400+0000,NaN,NaN,NaN,NaN,NaN,NaN
4,20231221T000000,20250224T012300,20250223T220731+0000,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,20220703083846+0000,20220703083846+0000,20220703083846+0000,20220408103245+0000,NaN,NaN,NaN,NaN
14,2024-06-03,NaN,2024-09-02T02:17:49+02:00,NaN,NaN,NaN,NaN,NaN,NaN
0,20230321000000,20250324075005,20250325060035,20250325060139,20241214020617,2.025033e+13,2.024121e+13,NaN,NaN
